In [ ]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it
    
from functools import partial
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.

import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm
from scipy.stats import ttest_ind

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    process_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies
)

from src.analysis.spec.wavelet_functions import get_uncorrected_wavelets, get_uncorrected_multitaper, get_sig_tfr_differences, plot_mask_pages

from src.analysis.utils.general_utils import (
    make_or_load_subjects_electrodes_to_ROIs_dict,
    get_good_data,
    get_sig_chans_per_subject,
    make_sig_electrodes_per_subject_and_roi_dict,
    calculate_total_electrodes,
    check_sampling_rates
)

import mne.time_frequency
from ieeg.calc.scaling import rescale
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)



['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\

#### 0. Load data.   
Need a way to load in the frequency information too, not just trials x channels x timepoints. Because I'm going to use frequency as a decoding feature too. For each electrode, for each training set, I think I can just mask the multitaper with the significant clusters and use that as the decoding feature, and then concatenate across electrodes to build the full training matrix.

In [31]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']

# params - these will become input variables once i functionalize this stuff
subjects = ['D0057']
signal_times = [-1.0, 1.5]
acc_trials_only = True
error_trials_only = False
stat_func = partial(ttest_ind, equal_var=False)
p_thresh = 0.05
ignore_adjacency = 1 # ignore the channels dimension for clusters, just find clusters over frequency and time
n_perm = 20
n_jobs = 1
freqs = np.arange(2, 200., 2.)
n_cycles = freqs / 2
return_itc = False
time_bandwidth=10 
spec_method = 'multitaper'
average=False
seed=None
tails=2

# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

task='GlobalLocal'
conditions = experiment_conditions.stimulus_switch_type_conditions # set this to whichever conditions you're running

stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = list(conditions.keys()) # get the condition names as a list

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [35]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [36]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057


get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [37]:
# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

In [ ]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 525


get number of sig and all electrodes per subject and across subjects

In [ ]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = all_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0057, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 17
Subject D0059, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 5
Subject D0063, ROI lpfc, Num of Sig Electrodes: 7, Num of All Electrodes: 36
Subject D0065, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 9
Subject D0069, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 1
Subject D0071, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 13
Subject D0077, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 0
Subject D0090, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 8
Subject D0094, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 28
Subject D0100, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 0
Subject D0102, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 5
Subject D0103, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 11
Subject D0107A, ROI lpfc, Num of Sig Electrodes: 1, Num of All Electrodes: 25
Subje

In [ ]:
# Example usage:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 44
Total number of lpfc electrodes across all subjects: 265
Total number of significant occ electrodes across all subjects: 18
Total number of occ electrodes across all subjects: 73


#### 1. For each electrode, make multitaper using all training trials, for both conditions to be compared (this can only do two conditions, can't do more rn)

In [ ]:
# TODO: Turn this cell into a function and implement some way of getting just accurate trials (whether using spec["Accuracy1.0"] or adding /Accuracy1.0 to the BIDS events)
# TODO: Need to loop over the trials and use some as training set, and have some held out as a test set that isn't used to find clusters. Maybe do this in the stats step though.

def make_subject_tfr_object(sub, good, condition_name, condition_dict, spec_method, signal_times, freqs, n_cycles, time_bandwidth, return_itc, n_jobs, average, epochs_root_file, acc_trials_only=False, error_trials_only=False):
    '''
    Calculates, saves, and returns a TFR object for a single subject and condition.

    This function computes the time-frequency representation from epochs data
    using either multitaper or wavelet analysis, optionally filtering for
    accurate or error trials. The resulting TFR object is saved to disk.

    Parameters
    ----------
    sub : str
        The subject identifier (e.g., '01').
    good : mne.Epochs
        The cleaned epochs data for the subject. Used for referencing
        bad channels and file paths.
    condition_name : str
        A descriptive name for the experimental condition (e.g., 'CorrectGo').
    condition_dict : dict
        A dictionary for the condition, which must contain the key "BIDS_events"
        with a list of BIDS event names.
    spec_method : str
        The spectral estimation method to use ('multitaper' or 'wavelets').
    signal_times : tuple of float
        The time window for TFR calculation in seconds, e.g., (-0.5, 1.5).
    freqs : np.ndarray
        An array of frequencies for the analysis.
    n_cycles : int or np.ndarray
        The number of cycles for the wavelet transform.
    time_bandwidth : float
        The time-bandwidth product for multitaper analysis (if used).
    return_itc : bool
        If True, calculates inter-trial coherence.
    n_jobs : int
        The number of parallel jobs to run for the calculation.
    average : bool
        Whether to average the TFR across trials.
    epochs_root_file : str
        The base name for the epochs root file (without the suffix).
    acc_trials_only : bool, optional
        If True, appends "/Accuracy1.0" to BIDS event names to select
        only accurate trials. Defaults to False.
    error_trials_only : bool, optional
        If True, appends "/Accuracy0.0" to BIDS event names to select
        only error trials. Defaults to False.

    Returns
    -------
    spec : mne.time_frequency.TFR
        The computed TFR object for the subject and condition.
    '''
    sub_spec_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, sub)
    if not os.path.exists(sub_spec_dir):
        os.makedirs(sub_spec_dir)

    # make signal wavelets
    BIDS_events = condition_dict["BIDS_events"] # extract the list of BIDS events

    if acc_trials_only:
        BIDS_events = [BIDS_event + "/Accuracy1.0" for BIDS_event in BIDS_events]
    elif error_trials_only:
        BIDS_events = [BIDS_event + "/Accuracy0.0" for BIDS_event in BIDS_events]
    if spec_method == 'multitaper':
        spec = get_uncorrected_multitaper(sub=sub, layout=layout, events=BIDS_events, times=signal_times, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=return_itc, n_jobs=n_jobs, average=average)
    elif spec_method == 'wavelets':
        spec = get_uncorrected_wavelets(sub=sub, layout=layout, events=BIDS_events, times=signal_times, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=return_itc, n_jobs=n_jobs, average=average)
    fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    spec.info['subject_info']['files'] = tuple(fnames)
    spec.info['bads'] = good.info['bads']

    # save the spec object
    save_path = os.path.join(sub_spec_dir, condition_name + '_' + spec_method + '_' + epochs_root_file)
    joblib.dump(spec, save_path)

    return spec

def make_subjects_tfr_objects(subjects, layout, conditions, spec_method, signal_times, freqs, n_cycles, time_bandwidth, return_itc, n_jobs, average, epochs_root_file="_epo.fif", acc_trials_only=False, error_trials_only=False, conditions_save_name="tfr_data"):
    '''
    Calculates and saves time-frequency representations (TFRs) for subjects.

    This function iterates through subjects and conditions to compute TFRs.
    It first checks if the TFR for a given subject and condition has already
    been computed and saved. If so, it loads the file; otherwise, it computes,
    saves, and then loads the TFR data. Finally, it saves a dictionary
    containing all TFR objects into a single file.

    Note: This function assumes that the underlying data for TFR calculation
    is not averaged across trials yet, allowing for trial-level operations
    (e.g., train-test splits) at a later stage.

    Parameters
    ----------
    subjects : list of str
        A list of subject identifiers (e.g., ['01', '02']).
    layout : bids.layout.BIDSLayout
        A BIDSLayout object pointing to the dataset root.
    conditions : dict
        A dictionary where keys are condition names (str) and
        values are dicts containing a list of BIDS event names under the key "BIDS_events".
        Example: {'ConditionA': {'BIDS_events': ['event/A1', 'event/A2']}}
    spec_method : str
        The spectral estimation method to use. Must be either
        'multitaper' or 'wavelets'.
    signal_times : tuple of float
        A tuple defining the time window for TFR
        calculation in seconds, e.g., (-0.5, 1.5).
    freqs : np.ndarray
        An array of frequencies for the analysis.
    n_cycles : int or np.ndarray
        The number of cycles for the wavelet transform.
    time_bandwidth : float
        The time-bandwidth product for multitaper analysis.
    return_itc : bool
        If True, calculates inter-trial coherence.
    n_jobs : int
        The number of parallel jobs to run.
    average : bool
        Whether to average the multitaper spectrogram across trials.
    epochs_root_file : str
        The base name for the epochs root file (without the suffix).
    acc_trials_only : bool, optional
        If True, appends "/Accuracy1.0" to BIDS
        event names to select only accurate trials. Defaults to False.
    error_trials_only : bool, optional
        If True, appends "/Accuracy0.0" to BIDS
        event names to select only error trials. Defaults to False.
    conditions_save_name (str, optional): The base name for the output file
        that aggregates all subjects' TFRs. Defaults to "tfr_data".

    Returns
    -------
    subjects_tfr_objects : dict
        A dictionary where keys are subject identifiers (str) and
        values are dicts containing TFR objects for each condition.
    '''
    subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')
    if not os.path.exists(subjects_tfr_objects_save_dir):
        os.makedirs(subjects_tfr_objects_save_dir)

    subjects_tfr_objects = {}

    for sub in subjects:
        subjects_tfr_objects[sub] = {}
        good = get_good_data(sub, layout)
        sub_spec_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, sub)

        for condition_name, condition_dict in conditions.items():
            # Define the path where the individual TFR object is or will be saved
            spec_save_path = os.path.join(sub_spec_dir, condition_name + '_' + spec_method + '_' + epochs_root_file)

            # Check if the TFR object already exists
            if os.path.exists(spec_save_path):
                print(f"Loading existing TFR object for sub-{sub}, condition: {condition_name}")
                spec = joblib.load(spec_save_path)
            else:
                print(f"Creating TFR object for sub-{sub}, condition: {condition_name}")
                spec = make_subject_tfr_object(sub=sub, good=good, condition_name=condition_name, condition_dict=condition_dict, spec_method=spec_method, signal_times=signal_times, freqs=freqs, n_cycles=n_cycles, time_bandwidth=time_bandwidth, return_itc=return_itc, n_jobs=n_jobs, average=average, epochs_root_file=epochs_root_file, acc_trials_only=acc_trials_only, error_trials_only=error_trials_only)

            subjects_tfr_objects[sub][condition_name] = spec

    # Save the aggregated dictionary of all subjects' TFR objects
    save_path = os.path.join(subjects_tfr_objects_save_dir, conditions_save_name + '_' + spec_method)
    joblib.dump(subjects_tfr_objects, save_path)

    return subjects_tfr_objects

In [ ]:
# if load_or_make_subjects_tfr_objects works, delete this direct call to make_subjects_tfr_objects
make_subjects_tfr_objects(
    subjects=subjects,
    layout=layout,
    conditions=conditions,
    spec_method=spec_method,
    signal_times=signal_times,
    freqs=freqs,
    n_cycles=n_cycles,
    time_bandwidth=time_bandwidth,
    return_itc=return_itc,
    n_jobs=n_jobs,
    acc_trials_only=acc_trials_only,
    error_trials_only=error_trials_only,
    conditions_save_name=conditions_save_name,
    average=average
)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file detected
Setting channel info str

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...


...

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 178 x 8243200 (4025.0 s), ~201 kB, data not loaded>
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 6198083  =      0.000 ...  3026.408 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Ac

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:1592: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_trials = mne.concatenate_epochs(all_trials_list)


Not setting metadata
189 matching events found
No baseline correction applied
NOTE: tfr_multitaper() is a legacy function. New code should use .compute_tfr(method="multitaper").


{'D0057': {'Stimulus_r': <Power Estimates from Epochs, multitaper method | 212 epochs × 175 channels × 99 freqs × 255 times, 2.0 - 198.0 Hz, -0.99 - 1.49 s, 6.98 GB>,
  'Stimulus_s': <Power Estimates from Epochs, multitaper method | 189 epochs × 175 channels × 99 freqs × 255 times, 2.0 - 198.0 Hz, -0.99 - 1.49 s, 6.22 GB>}}

In [25]:
# untested 7/6/25, will replace the direct call to make_subjects_tfr_objects
def load_or_make_subjects_tfr_objects(
    layout, 
    spec_method, 
    conditions_save_name, 
    subjects=None, 
    conditions=None, 
    signal_times=None, 
    freqs=None, 
    n_cycles=None, 
    time_bandwidth=None, 
    return_itc=None, 
    n_jobs=1, 
    average=False, 
    epochs_root_file="_epo.fif", 
    acc_trials_only=False, 
    error_trials_only=False
):
    """
    Loads pre-computed TFR objects, or makes them if they don't exist.

    This function checks for a file containing aggregated TFR objects. If the
    file exists, it is loaded. If not, this function calls
    `make_subjects_tfr_objects` to generate, save, and return the data.

    Parameters
    ----------
    layout : bids.layout.BIDSLayout
        The BIDS layout object for the dataset.
    spec_method : str
        The spectral estimation method used ('multitaper' or 'wavelets').
    conditions_save_name : str
        The base name of the file where the TFR objects are stored.
    
    ** The following parameters are required only if the TFR object file does not exist. **

    subjects : list of str, optional
        A list of subject identifiers (e.g., ['01', '02']).
    conditions : dict, optional
        A dictionary defining experimental conditions and their BIDS events.
    signal_times : tuple of float, optional
        Time window for TFR calculation, e.g., (-0.5, 1.5).
    freqs : np.ndarray, optional
        An array of frequencies for the analysis.
    n_cycles : int or np.ndarray, optional
        The number of cycles for the wavelet transform.
    time_bandwidth : float, optional
        The time-bandwidth product for multitaper analysis.
    return_itc : bool, optional
        If True, calculates inter-trial coherence.
    n_jobs : int, optional
        The number of parallel jobs to run. Defaults to 1.
    average : bool, optional
        Whether to average the TFR across trials. Defaults to False.
    epochs_root_file : str
        The base name for the epochs root file (without the suffix).
    acc_trials_only : bool, optional
        If True, select only accurate trials. Defaults to False.
    error_trials_only : bool, optional
        If True, select only error trials. Defaults to False.

    Returns
    -------
    dict
        A dictionary of TFR objects, structured as {subject: {condition: tfr_object}}.
    """
    subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')
    file_path = os.path.join(subjects_tfr_objects_save_dir, conditions_save_name + '_' + spec_method) # ugh it's not letting me rename the file to include an underscore so might have to remake it

    # 1. Check if the subjects_tfr_objects for this condition exists.
    if os.path.exists(file_path):
        # 2. If so, load it.
        print(f"Found existing TFR data. Loading from: {file_path}")
        subjects_tfr_objects = joblib.load(file_path)
    else:
        # 3. If not, make it.
        print(f"No existing TFR data found. Creating new file at: {file_path}")
        # Check if necessary arguments for creation are provided
        if any(arg is None for arg in [subjects, conditions, signal_times, freqs, n_cycles, time_bandwidth, return_itc]):
            raise ValueError("To create TFR objects, you must provide values for subjects, conditions, and all TFR parameters.")
        
        subjects_tfr_objects = make_subjects_tfr_objects(
            subjects=subjects,
            layout=layout,
            conditions=conditions,
            spec_method=spec_method,
            signal_times=signal_times,
            freqs=freqs,
            n_cycles=n_cycles,
            time_bandwidth=time_bandwidth,
            return_itc=return_itc,
            n_jobs=n_jobs,
            average=average,
            epochs_root_file=epochs_root_file,
            acc_trials_only=acc_trials_only,
            error_trials_only=error_trials_only,
            conditions_save_name=conditions_save_name
        )

    return subjects_tfr_objects

In [13]:
subjects_tfr_objects = load_or_make_subjects_tfr_objects(
    layout=layout,
    spec_method=spec_method,
    conditions_save_name=conditions_save_name,
    subjects=subjects,
    conditions=conditions,
    signal_times=signal_times,
    freqs=freqs,
    n_cycles=n_cycles,
    time_bandwidth=time_bandwidth,
    return_itc=return_itc,
    n_jobs=n_jobs,
    average=average,
    acc_trials_only=False 
)

Found existing TFR data. Loading from: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\multitaper\subjects_tfr_objects\stimulus_switch_type_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_1_subjectsmultitaper


#### 2. Find clusters that are significantly different between the two conditions, in the multitaper spectrogram  
do this for all channels per subject, and then also for all channels in an roi across subjects


In [39]:
def get_sig_tfr_differences_per_subject(
    subjects_tfr_objects: dict,
    condition_names: list[str],
    stat_func: callable,
    p_thresh: float = 0.05,
    p_cluster: float = None,
    n_perm: int = 1000,
    tails: int = 1,
    axis: int = 0,
    ignore_adjacency: int | tuple[int, ...] = 1,
    n_jobs: int = 1,
    seed: int = None):
    """
    Performs TFR statistical analysis for each subject individually.

    Parameters
    ----------
    subjects_tfr_objects : dict
        A dictionary structured as {subject_id: {condition_name: tfr_object}}.
    condition_names : list[str]
        A list of two condition names to be compared.
    stat_func: callable, optional
        The statistical function to use for significance testing. You should probably use partial(ttest_ind, equal_var=False).
    p_thresh : float
        The p-value threshold to use for determining significant time points.
    p_cluster : float, optional
        The p-value threshold to use for determining significant clusters.
    n_perm : int, optional
        The number of permutations to perform.
    tails : int, optional
        The number of tails to use. 1 for one-tailed, 2 for two-tailed.
    axis : int, optional
        The axis to perform the permutation test across. Also known as the
        observations axis
    ignore_adjacency : int or tuple of ints, optional
        The axis or axes to ignore when finding clusters. For example, if
        sig1.shape = (trials, channels, time), and you want to find clusters
        across time, but not channels, you would set ignore_adjacency = 1.
    n_jobs : int, optional
        The number of jobs to run in parallel. -1 for all processors. Default
        is -1.
    seed : int, optional
        The random seed to use for the permutation test. Default is None.

    Returns
    -------
    sub_masks : dict
        Dictionary mapping subject IDs to binary mask arrays indicating 
        significant differences between conditions.
    sub_pvals : dict
        Dictionary mapping subject IDs to p-values for each cluster found
        in the statistical comparison.
    """
    if len(condition_names) != 2:
        raise ValueError("This function requires exactly two conditions for comparison.")

    sub_masks = {}
    sub_pvals = {}
    cond1, cond2 = condition_names[0], condition_names[1]

    for sub, tfrs in subjects_tfr_objects.items():
        print(f"Processing statistics for subject: {sub}")
        
        mask, pvals = get_sig_tfr_differences(
            tfr_data_cond1=tfrs[cond1],
            tfr_data_cond2=tfrs[cond2],
            stat_func=stat_func,
            p_thresh=p_thresh,
            p_cluster=p_cluster,
            n_perm=n_perm,
            tails=tails,
            axis=axis,
            ignore_adjacency=ignore_adjacency,
            n_jobs=n_jobs,
            seed=seed
        )
        
        sub_masks[sub] = mask
        sub_pvals[sub] = pvals

    return (sub_masks, sub_pvals)

def get_sig_tfr_differences_per_roi(
    subjects_tfr_objects: dict,
    electrodes_per_subject_roi: dict,
    condition_names: list[str],
    stat_func: callable,
    p_thresh: float = 0.05,
    p_cluster: float = None,
    n_perm: int = 1000,
    tails: int = 1,
    axis: int = 0,
    ignore_adjacency: int | tuple[int, ...] = 1,
    n_jobs: int = 1,
    seed: int = None):
    """
    Performs TFR statistical analysis for each ROI by combining subjects.

    Parameters
    ----------
    subjects_tfr_objects : dict
        Dictionary of TFR data: {subject_id: {condition_name: tfr_object}}.
    electrodes_per_roi : dict
        Dictionary mapping ROIs to rois and electrodes: {roi_name: {subject_id: [elecs]}}.
    condition_names : list[str]
        A list of two condition names to compare.
    stat_func: callable, optional
        The statistical function to use for significance testing. You should probably use partial(ttest_ind, equal_var=False).
    p_thresh : float
        The p-value threshold to use for determining significant time points.
    p_cluster : float, optional
        The p-value threshold to use for determining significant clusters.
    n_perm : int, optional
        The number of permutations to perform.
    tails : int, optional
        The number of tails to use. 1 for one-tailed, 2 for two-tailed.
    axis : int, optional
        The axis to perform the permutation test across. Also known as the
        observations axis
    ignore_adjacency : int or tuple of ints, optional
        The axis or axes to ignore when finding clusters. For example, if
        sig1.shape = (trials, channels, time), and you want to find clusters
        across time, but not channels, you would set ignore_adjacency = 1.
    n_jobs : int, optional
        The number of jobs to run in parallel. -1 for all processors. Default
        is -1.
    seed : int, optional
        The random seed to use for the permutation test. Default is None.

    Returns
    -------
    roi_masks : dict
        Dictionary mapping ROI names to concatenated binary mask arrays.
        For each ROI, masks from all subjects with electrodes in that ROI
        are concatenated along axis 0 (subjects dimension), resulting in
        a combined mask array with shape (n_subjects_in_roi, *original_dims).
    roi_pvals : dict
        Dictionary mapping ROI names to concatenated p-value arrays.
        For each ROI, p-values from all subjects with electrodes in that ROI
        are concatenated along axis 0, matching the structure of roi_masks.
        Empty arrays are returned for ROIs with no subject data.
    """
    if len(condition_names) != 2:
        raise ValueError("This function requires exactly two conditions for comparison.")

    roi_masks = {}
    roi_pvals = {}
    cond1, cond2 = condition_names[0], condition_names[1]

    for roi, subjects_in_roi in electrodes_per_subject_roi.items():
        print(f"Processing statistics for ROI: {roi}")
        
        subject_masks_for_roi = []
        subject_pvals_for_roi = []

        for sub, tfrs in subjects_tfr_objects.items():
            elecs = subjects_in_roi.get(sub, [])
            if not elecs:
                continue

            mask, pvals = get_sig_tfr_differences_per_subject(
                tfr_data_cond1=tfrs[cond1],
                tfr_data_cond2=tfrs[cond2],
                stat_func=stat_func,
                elecs_to_pick=elecs,
                p_thresh=p_thresh,
                p_cluster=p_cluster,
                n_perm=n_perm,
                tails=tails,
                axis=axis,
                ignore_adjacency=ignore_adjacency,
                n_jobs=n_jobs,
                seed=seed
            )

            subject_masks_for_roi.append(mask)
            subject_pvals_for_roi.append(pvals)

        if subject_masks_for_roi:
            roi_masks[roi] = np.concatenate(subject_masks_for_roi, axis=0)
            roi_pvals[roi] = np.concatenate(subject_pvals_for_roi, axis=0)
        else:
            roi_masks[roi] = np.array([])
            roi_pvals[roi] = np.array([])

    return roi_masks, roi_pvals

In [ ]:
subjects_tfr_objects['D0057']['Stimulus_r'].average().plot(picks='RAI1')

No baseline correction applied


In [44]:
# For per-subject analysis (no electrode filtering needed)
sig_elec_masks_per_subject, sig_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

all_elec_masks_per_subject, all_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# For per-ROI analysis (with electrode filtering)
sig_elec_masks_per_roi, sig_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=sig_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

all_elec_masks_per_roi, all_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=all_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

Processing statistics for subject: D0057
stat_func returns a tuple. Taking the first element


c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\scipy\stats\_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\scipy\stats\_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    4.2s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    8.5s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:   10.0s


KeyboardInterrupt: 

plot these clusters (code not working yet, copied over from decoding.ipynb)

In [ ]:
# TODO: Fix the plotting code below, it's copied from wavelet_differences. I should make it so it can take in the sig_elecs_mask, all_elecs_mask for specific rois or for all channels in a subject
# TODO: If plotting per roi, then need to append the subject for each electrode to the electrode name. Check vis/plot_subjects.ipynb for an example.
# plot

first_sub = subjects[0]
first_condition = list(subjects_tfr_objects.keys())[0]
ch_names, times, freqs = subjects_tfr_objects[first_condition][first_sub].ch_names, subjects_tfr_objects[first_condition][first_sub].times, subjects_tfr_objects[first_condition][first_sub].freqs  # list of channel names

# Now plot the mask pages:
for sub in subjects:
    sig_elecs_mask = sig_elec_masks_per_subject[sub]
    sig_elecs_mask_pages = plot_mask_pages(sig_elecs_mask,
                    ch_names,
                    times=times,
                    freqs=freqs,
                    channels_per_page=60,
                    grid_shape=(6, 10),
                    cmap=parula_map,
                    title_prefix=f"{sub} ",
                    log_freq=True,
                    show=False)

    # Save each page as a separate figure file:
    for i, fig in enumerate(sig_elecs_mask_pages):
        fig_name = f"{sub}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    all_elecs_mask = all_elec_masks_per_subject[sub]
    all_elecs_mask_pages = plot_mask_pages(all_elecs_mask,
                        ch_names,
                        times=times,
                        freqs=freqs,
                        channels_per_page=60,
                        grid_shape=(6, 10),
                        cmap=parula_map,
                        title_prefix=f"{sub} ",
                        log_freq=True,
                        show=False)

    # Save each page as a separate figure file:
    for i, fig in enumerate(all_elecs_mask_pages):
        fig_name = f"{sub}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
        fig_pathname = os.path.join(save_dir, fig_name)
        fig.savefig(fig_pathname, bbox_inches='tight')
        print("Saved figure:", fig_name)

    for roi in rois:
        sig_elecs_roi_mask = sig_elec_masks_per_roi[sub][roi]
        sig_elecs_roi_mask_pages = plot_mask_pages(sig_elecs_roi_mask,
                        ch_names,
                        times=times,
                        freqs=freqs,
                        channels_per_page=60,
                        grid_shape=(6, 10),
                        cmap=parula_map,
                        title_prefix=f"{sub} {roi} ",
                        log_freq=True,
                        show=False)

        # Save each page as a separate figure file:
        for i, fig in enumerate(sig_elecs_roi_mask_pages):
            fig_name = f"{sub}_{roi}_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)

        all_elecs_roi_mask = all_elec_masks_per_roi[sub][roi]
        all_elecs_roi_mask_pages = plot_mask_pages(all_elecs_roi_mask,
                        ch_names,
                        times=times,
                        freqs=freqs,
                        channels_per_page=60,
                        grid_shape=(6, 10),
                        cmap=parula_map,
                        title_prefix=f"{sub} {roi} ",
                        log_freq=True,
                        show=False)

        # Save each page as a separate figure file:
        for i, fig in enumerate(all_elecs_roi_mask_pages):
            fig_name = f"{sub}_{roi}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
            fig_pathname = os.path.join(save_dir, fig_name)
            fig.savefig(fig_pathname, bbox_inches='tight')
            print("Saved figure:", fig_name)
            
# # get the mean differences themselves and plot them
# mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
# mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

# # Now, plot the mean differences directly:
# congruency_mean_diff_pages = plot_mask_pages(
#     mean_diff_inc_vs_con,
#     incongruent_spec.ch_names,
#     times=incongruent_spec.times,
#     freqs=incongruent_spec.freqs,
#     grid_shape=(6, 10),
#     cmap=parula_map,  # play with color maps
#     title_prefix=f"{sub} Mean Inc-Con Diff: ",
#     log_freq=True,
#     show=False
# )

# # Save each page as a separate figure file:
# for i, fig in enumerate(congruency_mean_diff_pages):
#     if rescaled:
#         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
#     else:
#         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
#     fig_pathname = os.path.join(save_dir, fig_name)
#     fig.savefig(fig_pathname, bbox_inches='tight')
#     print("Saved figure:", fig_name)

Need to figure out how to use the masks to index the subjects_tfr_objects. The masks should be 3D (frequencies x channels x times, or some order of this), and the subjects_tfr_objects should be 4D (trials x frequencies x channels x time, or some order of this).  

So there should be a way to use the mask to grab the appropriate frequencies x channels x times from subjects_tfr_objects for each subject, 



#### 3. Train a decoder on just the significant time-frequency clusters, test on the test trials

#### 4. Repeat but with new test trials (cross-validate)

#### 5. Figure out a way of plotting this. This is univariate approach, make things modular so I can do multivariate later once I figure that out.